In [21]:
%load_ext autoreload
%autoreload 2
import numpy as np
import Plot as plot
import math
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
import time
import datetime
import pickle
import os
import scipy.spatial
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib as plt
import logging
from scipy import signal
import learningAlgs as classImportLA
import dataManipulation as dataMan
from itertools import permutations
import importlib
from datetime import timedelta
from multiprocessing import Pool
import multiprocessing
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.cluster import KMeans
import timeIntervalPlotter as intervalPlotter
import pysal
import warnings
import lumping_traditional as oldLumping
import boundaryFull_SS_WeightedLumping as WLumping
from importlib import reload
from scipy.stats import rayleigh
import dataManipulation as dataMan
import matplotlib.pyplot as plt
import processData as processData
import tensorflow as tf
# import tensorflow_probability as tfp
import random
from random import shuffle

warnings.filterwarnings('always')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import pkg_resources
pkg_resources.get_distribution("tensorflow").version

'1.14.0'

# Fetching Feature Engineered Data

In [5]:
data = pd.read_pickle("30min_featureEngineer.pickle")

# Preparing Data for Training and Testing

In [6]:
def classifying(CU, boundaries):
    occupiedBandwidth = (CU / 255) * 100
    for i in range(len(boundaries)):
        if occupiedBandwidth <= boundaries[i]:
            return i

In [7]:
def dataPreparation(data, timeIndexes, minuteSplit, boundaries):
    from sklearn.utils import shuffle
    warnings.filterwarnings('always')
    reg = "l2"
    solvers = "lbfgs"
    clf = LogisticRegression(penalty = reg, max_iter = 100000, random_state = 0,
                             solver = solvers , multi_class = 'multinomial')
    accuracyValue = 0
    numOfElements = 0
    f1scoreValue = 0
    precisionValue = 0
    recallValue = 0
    prevRowTrain = np.inf
    prevCU = np.inf

    sampleIntervals = 6 #seconds
    minuteSplit = 30 #minutes
    numOfSamples = minuteSplit * 60 / sampleIntervals
    # numberOfDays = len(numOfDays)
    days = np.zeros(7)
    numOfThirtyMinsPerDay = np.zeros(int((24 * 60) / minuteSplit)) #in this case 48
#     which6SecondsPerPeriod = np.zeros(int(minuteSplit * 60 / sampleIntervals)) #in this case 300
    prevRowTrain = np.inf
    prevCU = np.inf

    XArraysForLearning = []
    YArraysForLearning = []
    XArraysForTesting = []
    YArraysForTesting = []

    x = timeIndexes
    wholeDataFrame = data.loc[(data["timeIndex"] == x)].copy()
    
    wholeDataFrame["cuClass"] = wholeDataFrame["CU"].apply(lambda x: classifying(x, boundaries))

    

    stackCounter = 0
    prevCU = 0
    prev2CU = 0
    prev3CU = 0
    prev4CU = 0
    prev5CU = 0
    print("start training set generation")
    #number of features are: prevCU + 48 correlations + 1 logDiff
    numberOfFeatures = len(boundaries) + 48 + 1 
    
    XArraysForLearning = np.zeros(shape=(len(wholeDataFrame), numberOfFeatures))
    YArraysForLearning = np.zeros(shape=(len(wholeDataFrame), len(boundaries)))
#     print(len(trainingDataFrame))
    counter = 0
    pandasIndexCounter = 0
    for index, row in wholeDataFrame.iterrows():
        XArraysForLearning[pandasIndexCounter, prevCU] = 1
        for corrs in range(48):
            XArraysForLearning[pandasIndexCounter, len(boundaries) + corrs] = row["corr" + str(corrs + 1)]

        XArraysForLearning[pandasIndexCounter, len(boundaries) + 48] = row["normalLogDiff"]
        YArraysForLearning[pandasIndexCounter, row["cuClass"]] = 1
#         print(XArraysForLearning[pandasIndexCounter])

        which6SecondsPerPeriod = 0
    
        prevCU = row["cuClass"]
        pandasIndexCounter += 1

    XArraysForLearning, YArraysForLearning = shuffle(XArraysForLearning, YArraysForLearning, random_state=0)
    XArraysForTraining = XArraysForLearning[:int(0.8*len(XArraysForLearning))]
    YArraysForTraining = YArraysForLearning[:int(0.8*len(YArraysForLearning))]
    XArraysForTesting = XArraysForLearning[int(0.8*len(XArraysForLearning)):]
    YArraysForTesting = YArraysForLearning[int(0.8*len(YArraysForLearning)):]
    print(XArraysForTraining.shape[0])
    print(XArraysForTesting.shape[0])


    return XArraysForTraining, YArraysForTraining, XArraysForTesting, YArraysForTesting, boundaries


# TensorFlow Function Running

In [32]:
def tensorFlowLossFunction(
    lossFuncBool, XArraysForLearning, YArraysForLearning, XArraysForTesting, YArraysForTesting, boundaries):
    
    batch_size = 64
    learning_rate = 0.01
    beta = 0.1
    numOfEpochs = 500
    graph = tf.Graph()
    with graph.as_default():
        x = tf.placeholder(tf.float32, shape = (batch_size, XArraysForLearning.shape[1]))
        y_ = tf.placeholder(tf.float32, shape = (batch_size, YArraysForLearning.shape[1]))
#         print(weights.dtype)
#         print(weights.shape)
#         print(XArraysForLearning.shape[1], YArraysForLearning.shape[1])
#         multiDistrib = np.vstack((mult, mult))
#         for i in range(batch_size - 2):
#             multiDistrib = np.vstack((multiDistrib, mult))
        W = tf.Variable(tf.truncated_normal([XArraysForLearning.shape[1], YArraysForLearning.shape[1]], seed = 0), name="weights", dtype=tf.float32)
        b = tf.Variable(tf.truncated_normal([YArraysForLearning.shape[1]], seed = 0), dtype=tf.float32)

        tf_test_dataset64 = tf.constant(XArraysForTesting)
        tf_test_dataset = tf.cast(tf_test_dataset64, tf.float32)


        beta = 0.05
        logits = tf.matmul(x, W)
        train_prediction = tf.nn.softmax(logits)
        # train_prediction = tf.nn.softmax_cross_entropy_with_logits_v2(labels = y_, logits = logits)
        test_prediction = tf.nn.softmax(tf.add(tf.matmul(tf_test_dataset, W),b))

        # x = XArraysForLearning[0:(0 + batch_size), :]
        # y_ = tf.Variable(YArraysForLearning[0:(0 + batch_size), :])

        # loss = assymetricLossFunction(train_prediction, y_, boundaries)
        # loss = assymetricLossFunction(train_prediction, y_, boundaries)
        
        if lossFuncBool == 0:
            loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = y_)
        
        elif lossFuncBool == 1:
            loss = assymetricLossFunction(train_prediction, y_, boundaries)
#         dist = tfp.distributions.Multinomial(total_count=1, logits=logits)
#         loss = loss - (dist.log_prob(mult))
        # regularizer = tf.nn.l2_loss(W)
        # loss = tf.reduce_mean(loss + beta * regularizer)
        # loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits = train_prediction, labels = y_)
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

        prevAcc = 0

    with tf.Session(graph=graph) as session:
        initializer = tf.contrib.layers.xavier_initializer()
        tf.set_random_seed(0)
        W = tf.Variable(initializer([XArraysForLearning.shape[1], YArraysForLearning.shape[1]]))
        b = tf.Variable(initializer([YArraysForLearning.shape[1]]))       
        tf.global_variables_initializer().run()
#         print(W.eval())
        
        print("Initialized")

        numberOfBatchIteration = int(XArraysForLearning.shape[0] / batch_size)
        restOfData = XArraysForLearning.shape[0] % batch_size
        if restOfData != 0:
            numberOfBatchIteration += 1

        accuracy_result = 0
        accuracy_earlyStop = 0
        earlyStoppingCounter = 0
        
        for epoch in range(numOfEpochs):
            accuracyValue = 0
            lossValue = 0
            totalBatch = 0
            i = 0
            randomize = np.arange(XArraysForLearning.shape[0])
            random.Random(epoch).shuffle(randomize)
            XArraysForLearning = XArraysForLearning[randomize]
            YArraysForLearning = YArraysForLearning[randomize]

            for iteration in range(numberOfBatchIteration):
                if (iteration == numberOfBatchIteration - 1) and restOfData != 0:
                    break
                    batch_data = XArraysForLearning[i:, :]
                    batch_labels = YArraysForLearning[i:, :]

                else:
                    batch_data = XArraysForLearning[i:(i + batch_size), :]
                    batch_labels = YArraysForLearning[i:(i + batch_size), :]

                    i += batch_size


                feed_dict = {x : batch_data, y_ : batch_labels}
                _, predictions, l = session.run([optimizer, train_prediction, loss], feed_dict=feed_dict)

#                 print(l)
                lossValue *= (iteration)
                lossValue += (np.sum(l))
                lossValue /= (iteration + 1)
                totalBatch += batch_size
                # print(accuracy(batch_data, batch_labels))
                accuracyValue += accuracy(predictions, batch_labels) * batch_size
#                 if accuracy(test_prediction.eval(), YArraysForTesting) < prevAcc and earlyStoppingCounter == 100:
#                     print(prevAcc)
#                     break
                
#                 elif accuracy(test_prediction.eval(), YArraysForTesting) < prevAcc:
#                     earlyStoppingCounter += 1
                
#                 elif accuracy(test_prediction.eval(), YArraysForTesting) >= prevAcc:
#                     earlyStoppingCounter = 0

            totalAccuracy = accuracyValue/totalBatch
#             print(session.run(W))
            print("epoch ", epoch, totalAccuracy)
#             if accuracy(test_prediction.eval(), YArraysForTesting) > accuracy_earlyStop:
#                 accuracy_earlyStop = accuracy(test_prediction.eval(), YArraysForTesting)

#             else:
#                 earlyStoppingCounter += 1

#             if earlyStoppingCounter == 50:
#                 break
            



        # print(session.run(W))
        predictionResult = test_prediction.eval()
        accResult = accuracy(predictionResult , YArraysForTesting)
        penaltyValue = assymetricPredictionScore(predictionResult, YArraysForTesting, boundaries)
#         print(W.eval())
#         if accuracy_earlyStop > accResult:
#             accuracy_result = accuracy_earlyStop
#         else:
        accuracy_result = accResult
            
        correctLableIndex = np.argmax(YArraysForTesting, 1)
        predictionIndex = np.argmax(predictionResult, 1)
        
        precision = precision_score(correctLableIndex, predictionIndex, average='weighted')
        recall = recall_score(correctLableIndex, predictionIndex, average='weighted')
        f1Score = f1_score(correctLableIndex, predictionIndex, average='weighted')            
        
        return accuracy_result, penaltyValue, precision, recall, f1Score
            
    
def assymetricPredictionScore(predictedLables, trueLables, boundaries):
    xAxisPoints = np.linspace(rayleigh.ppf(0.01), rayleigh.ppf(0.99), 338)
    #number of overal datapoints must stay the same all the time
    maxState = 338

    inverseDistrib = max(rayleigh.pdf(xAxisPoints)) - rayleigh.pdf(xAxisPoints)
    minState = np.argmin(inverseDistrib)

    underUtilizedSum = 0
    overUtilizedSum = 0
    numberOfUnderUtilizedStates = minState - 0
    numberOfOverUtilizedStates = maxState - minState

    xAxisPoints -= xAxisPoints[np.argmin(inverseDistrib)]

    underUtilVal = numberOfUnderUtilizedStates / 100
    overUtilVal = numberOfOverUtilizedStates / 100

    correctLableIndex = np.argmax(trueLables, 1)
    predictionIndex = np.argmax(predictedLables, 1)

    diffPercentage = np.zeros(shape = (predictedLables.shape))

    penalties = np.zeros(shape = (predictedLables.shape))

    for index in range(predictedLables.shape[0]):
        diffPercentage[index] = boundaries[correctLableIndex[index]] - boundaries[:]


    for i in range(diffPercentage.shape[0]):
        for j in range(diffPercentage.shape[1]):
            if diffPercentage[i][j] > 0:
                penalties[i][j] = inverseDistrib[minState + math.floor
                                                  (diffPercentage[i][j] * overUtilVal)]
            else:
                penalties[i][j] = inverseDistrib[minState + math.floor
                                                  (diffPercentage[i][j] * underUtilVal)]

    sumOfPenalty = 0
    for i in range(predictionIndex.shape[0]):
        penalties[i] = penalties[i] / np.sum(penalties[i])
        sumOfPenalty += penalties[i][predictionIndex[i]]

    return sumOfPenalty
    
    
def accuracy(predictedLables, trueLables):
    import sys
    np.set_printoptions(threshold=sys.maxsize)
    correctLableIndex = np.argmax(trueLables, 1)
    predictionIndex = np.argmax(predictedLables, 1)
    errors = [0 for i in range(trueLables.shape[1])]
    corrects = [0 for i in range(trueLables.shape[1])]
    for i in range(len(correctLableIndex)):
        if correctLableIndex[i] != predictionIndex[i]:
            errors[correctLableIndex[i]] += 1
            corrects[predictionIndex[i]] += 1

    acc = np.float64(np.sum(correctLableIndex == predictionIndex)/predictedLables.shape[0])
    return acc
    
    
def assymetricLossFunction(prediction, correctLable, boundaries):
    xAxisPoints = np.linspace(rayleigh.ppf(0.01), rayleigh.ppf(0.99), 338)
    #number of overal datapoints must stay the same all the time
    maxState = 338
    inverseDistrib = max(rayleigh.pdf(xAxisPoints)) - rayleigh.pdf(xAxisPoints)
    inverseDistrib = tf.constant(inverseDistrib)
    xAxisPoints -= xAxisPoints[np.argmin(inverseDistrib)]
    minState = np.argmin(inverseDistrib)
    numberOfOverUtilizedStates = maxState - minState
    numberOfUnderUtilizedStates = minState
    minState = tf.constant(minState, tf.float32)
    numberOfOverUtilizedStates = tf.constant(numberOfOverUtilizedStates, tf.float32)
    numberOfUnderUtilizedStates = tf.constant(numberOfUnderUtilizedStates, tf.float32)

    underUtilVal = numberOfUnderUtilizedStates / 100
    overUtilVal = numberOfOverUtilizedStates / 100


    boundaries = tf.constant(boundaries, tf.float32)
    correctLableIndex = tf.argmax(correctLable, 1)

    diffPercentage = []
    for index in range(correctLableIndex.shape[0]):
        diffPercentage.append(boundaries[correctLableIndex[index]] - boundaries[:])

    diffPercentage = tf.stack(diffPercentage)

    penalties = []

    counter = 0
    for i in range(diffPercentage.shape[0]):
        for j in range(diffPercentage.shape[1]):
            counter += 1

            penalties.append(tf.cond(
                    tf.greater(diffPercentage[i][j], 0),
                    lambda: inverseDistrib[tf.dtypes.cast(minState + tf.math.floor
                                                                   (tf.math.scalar_mul(diffPercentage[i][j],
                                                                                       overUtilVal)), tf.int32)],
                    lambda: inverseDistrib[tf.dtypes.cast(minState + tf.math.floor
                                                                   (tf.math.scalar_mul
                                                                    (diffPercentage[i][j], underUtilVal))
                                                                   , tf.int32)]
                    ))

    penalties = tf.stack(penalties)
    penalties = tf.dtypes.cast(penalties, tf.float32)
    penalties = tf.reshape(penalties, diffPercentage.shape)
    penalties = penalties / tf.norm(penalties)

    # weights = tf.reduce_sum(penalties * (1-prediction), axis=1)
    weights = (1 - penalties) * prediction
    weights = weights / tf.norm(weights)
    # print(correctLable)
    # print(prediction)
    # print(penalties)
    loss = tf.losses.softmax_cross_entropy(onehot_labels = correctLable, logits = weights)
    # weighted_losses = tf.nn.softmax_cross_entropy_with_logits_v2(labels = penalties, logits = prediction)
    # loss = tf.reduce_sum(weighted_losses)
    # loss = tf.reduce_sum(penalties * prediction)
    return tf.reduce_mean(loss)

# Fetching Lumped Matrices with their Bandwidths

In [9]:
name = "/home/sepehr/thesis/APDataML/normalBoundaries_30min.pickle"
with open(name, 'rb') as handle:
    normalLumping = pickle.load(handle)

In [11]:
percentageArray = []
for i in range(48):
    name = "/home/sepehr/thesis/APDataML/pickles/normal_lumping_percentage_" + str(i) + ".pickle"
    with open(name, 'rb') as handle:
        percentageArray.append(pickle.load(handle))

In [22]:
accuracyResult = [np.inf for i in range(48)]
penaltyResult = [np.inf for i in range(48)]
precisionResult = [np.inf for i in range(48)]
recallResult = [np.inf for i in range(48)]
f1ScoreResult = [np.inf for i in range(48)]
boundariesResult = [np.inf for i in range(48)]
lumpAproxResult = [np.inf for i in range(48)]
lumpErrorResult = [np.inf for i in range(48)]


testDataFrame = {"accuracy": accuracyResult, "penalty": penaltyResult, "precision": precisionResult, 
                 "recall": recallResult, "f1Score": f1ScoreResult, "boundaries": boundariesResult, 
                 "lumpAprox": lumpAproxResult, "lumpError": lumpErrorResult}

resultDataFrameWithPenalty = pd.DataFrame(testDataFrame)
resultDataFrameWithPenalty = resultDataFrameWithPenalty.astype('object')

resultDataFrameNoPenalty = pd.DataFrame(testDataFrame)
resultDataFrameNoPenalty = resultDataFrameNoPenalty.astype('object')

In [25]:
warnings.filterwarnings('ignore')
for timeIndex in range(5):
    if len(normalLumping[timeIndex]) == 0:
        continue
    
    print(timeIndex)
    boundaries = np.array([])
    for i in range(1, len(normalLumping[0][0][3])):
        bound = normalLumping[0][0][3][i]
        counter = 0
        for j in range(len(percentageArray[0])):
            if percentageArray[0][j][2] == False:
                counter += 1
            if counter == bound:
                boundaries = np.append(boundaries, percentageArray[0][j][1])
                break
    boundaries = np.append(boundaries,100)
    
    XArraysForLearning, YArraysForLearning, XArraysForTesting, YArraysForTesting, boundaries = dataPreparation(
        data, timeIndex, 30, boundaries)

    print("starting tensor")
    lossFunctionBoolean = 0
    accuracy_result, penaltyValue, precision, recall, f1Score = tensorFlowLossFunction(
        lossFunctionBoolean, XArraysForLearning, YArraysForLearning, XArraysForTesting, 
        YArraysForTesting, boundaries)
    
    resultDataFrameNoPenalty.loc[timeIndex]["accuracy"] = accuracy_result
    resultDataFrameNoPenalty.loc[timeIndex]["penalty"] = penaltyValue
    resultDataFrameNoPenalty.loc[timeIndex]["precision"] = precision
    resultDataFrameNoPenalty.loc[timeIndex]["recall"] = recall
    resultDataFrameNoPenalty.loc[timeIndex]["f1Score"] = f1Score
    resultDataFrameNoPenalty.loc[timeIndex]["boundaries"] = boundaries
    resultDataFrameNoPenalty.loc[timeIndex]["lumpAprox"] = normalLumping[timeIndex][0][0]
    resultDataFrameNoPenalty.loc[timeIndex]["lumpError"] = normalLumping[timeIndex][0][1]
    
    
    lossFunctionBoolean = 1
    accuracy_result, penaltyValue, precision, recall, f1Score = tensorFlowLossFunction(
        lossFunctionBoolean, XArraysForLearning, YArraysForLearning, XArraysForTesting, 
        YArraysForTesting, boundaries)
    
    
    resultDataFrameWithPenalty.loc[timeIndex]["accuracy"] = accuracy_result
    resultDataFrameWithPenalty.loc[timeIndex]["penalty"] = penaltyValue
    resultDataFrameWithPenalty.loc[timeIndex]["precision"] = precision
    resultDataFrameWithPenalty.loc[timeIndex]["recall"] = recall
    resultDataFrameWithPenalty.loc[timeIndex]["f1Score"] = f1Score
    resultDataFrameWithPenalty.loc[timeIndex]["boundaries"] = boundaries
    resultDataFrameWithPenalty.loc[timeIndex]["lumpAprox"] = normalLumping[timeIndex][0][0]
    resultDataFrameWithPenalty.loc[timeIndex]["lumpError"] = normalLumping[timeIndex][0][1]
    
    print(resultDataFrameNoPenalty.loc[timeIndex])
    

0
start training set generation
19075
4769
starting tensor
Initialized
Initialized


/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



1
start training set generation
18820
4706
starting tensor
Initialized
Initialized


KeyboardInterrupt: 

In [30]:
print(resultDataFrameNoPenalty.loc[0])

accuracy                                               0.954707
penalty                                                 8.60518
precision                                              0.949913
recall                                                 0.950304
f1Score                                                 0.95003
boundaries    [23.13725490196077, 34.11764705882348, 50.1960...
lumpAprox                                                     1
lumpError                                                 40.08
Name: 0, dtype: object


In [33]:
accuracy_result, penaltyValue, precision, recall, f1Score = tensorFlowLossFunction(
    1, XArraysForLearning, YArraysForLearning, XArraysForTesting, 
    YArraysForTesting, boundaries)

Initialized
epoch  0 0.8470450680272109
epoch  1 0.9311224489795918
epoch  2 0.935905612244898


KeyboardInterrupt: 

In [34]:
accuracy_result, penaltyValue, precision, recall, f1Score = tensorFlowLossFunction(
    1, XArraysForLearning, YArraysForLearning, XArraysForTesting, 
    YArraysForTesting, boundaries)

Initialized
epoch  0 0.8470450680272109
epoch  1 0.9311224489795918
epoch  2 0.935905612244898
epoch  3 0.9357993197278912


KeyboardInterrupt: 